In [12]:
# import the necessary libraries
import cv2
import os
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_column', 1000)

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D,\
BatchNormalization
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

from unet.augment import random_hsv, random_shift_scale_rotate, random_horizontal_flip
from unet.loss import dice_coeff, dice_loss, bce_dice_loss
from unet.train_generator import train_generator
from unet.valid_generator import valid_generator
from unet.layers import unet_encode, unet_maxpool, unet_decode
from unet.network import unet
from unet.rle import run_length_encode

from tqdm import tqdm

In [14]:
# setting global parameters
input_size = 128
input_shape = (128, 128, 3)
num_classes = 1
start_filters = 64
center_filters = 1024
learning_rate = 0.0001
max_epochs = 30
batch_size = 4
orig_width = 1918
orig_height = 1280
threshold = 0.5

In [3]:
# summary of u-net architecture
model = unet(input_shape = input_shape,
            num_classes = num_classes,
            start_filters = start_filters,
            center_filters = center_filters,
            learning_rate = learning_rate)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 128, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

### Train test split by 80 / 20 ratio

In [4]:
traindf = pd.read_csv('train_masks.csv')
img_id = traindf['img'].map(lambda x: x.split('.')[0])

train_id, valid_id = train_test_split(img_id, test_size = 0.2, random_state = 42)
print('Training set with {} images'.format(len(train_id)), '\n')
print('Validation set with {} images'.format(len(valid_id)))

Training set with 4070 images 

Validation set with 1018 images


### Network Callbacks

In [5]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='min'),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='min'),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/128_best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=1,
                             mode='min'),
             TensorBoard(log_dir='logs')]

C:\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


### Training the U-Net network

In [6]:
train_generator = train_generator(input_size = input_size,
                                 train_id = train_id,
                                 batch_size = batch_size)

valid_generator = valid_generator(input_size = input_size,
                                 valid_id = valid_id,
                                 batch_size = batch_size)

In [7]:
history = model.fit_generator(generator = train_generator,
                    steps_per_epoch = np.ceil(float(len(train_id)) / float(batch_size)),
                    epochs = max_epochs,
                    verbose = 2,
                    callbacks = callbacks,
                    validation_data = valid_generator,
                    validation_steps = np.ceil(float(len(valid_id)) / float(batch_size)))

Epoch 1/30
 - 383s - loss: 0.1366 - dice_coeff: 0.9177 - val_loss: 0.0579 - val_dice_coeff: 0.9682

Epoch 00001: val_loss improved from inf to 0.05789, saving model to weights/128_best_weights.hdf5
Epoch 2/30
 - 375s - loss: 0.0433 - dice_coeff: 0.9769 - val_loss: 0.0330 - val_dice_coeff: 0.9825

Epoch 00002: val_loss improved from 0.05789 to 0.03304, saving model to weights/128_best_weights.hdf5
Epoch 3/30
 - 375s - loss: 0.0335 - dice_coeff: 0.9826 - val_loss: 0.0249 - val_dice_coeff: 0.9870

Epoch 00003: val_loss improved from 0.03304 to 0.02488, saving model to weights/128_best_weights.hdf5
Epoch 4/30
 - 371s - loss: 0.0304 - dice_coeff: 0.9843 - val_loss: 0.0228 - val_dice_coeff: 0.9879

Epoch 00004: val_loss improved from 0.02488 to 0.02280, saving model to weights/128_best_weights.hdf5
Epoch 5/30
 - 364s - loss: 0.0288 - dice_coeff: 0.9851 - val_loss: 0.0207 - val_dice_coeff: 0.9891

Epoch 00005: val_loss improved from 0.02280 to 0.02068, saving model to weights/128_best_weights

### Classifying our output masks

In [10]:
testdf = pd.read_csv('sample_submission.csv')
img_test_id = testdf['img'].map(lambda s: s.split('.')[0])

ids = []

for id in img_test_id:
    ids.append('{}.jpg'.format(id))
print('Test set with {} images'.format(len(ids)))

Test set with 100064 images


In [15]:
rles = []

model.load_weights(filepath = 'weights/128_best_weights.hdf5')

print('[Info] Classifying on {} test images with {} batch_size...'.format(len(img_test_id), batch_size))
for start in tqdm(range(0, len(img_test_id), batch_size)):
    x_batch = []
    end = min(start + batch_size, len(img_test_id))
    test_batch = img_test_id[start:end]
    for id in test_batch.values:
        img = cv2.imread('test/{}.jpg'.format(id))
        img = cv2.resize(img, (input_size, input_size))
        x_batch.append(img)
    x_batch = np.array(x_batch, np.float32) / 255
    preds = model.predict_on_batch(x_batch)
    preds = np.squeeze(preds, axis=3)
    for pred in preds:
        prob = cv2.resize(pred, (orig_width, orig_height))
        mask = prob > threshold
        rle = run_length_encode(mask)
        rles.append(rle)

[Info] Classifying on 100064 test images with 4 batch_size...


100%|██████████| 25016/25016 [1:40:17<00:00,  3.79it/s]


In [19]:
print("Generating submission file...",'\n')
df = pd.DataFrame({'img': ids, 'rle_mask': rles})
df.to_csv('submit/128_submission.csv', index=False)
print('Done!')

Generating submission file... 

Done!
